In [ ]:
import boto3
import pandas as pd

s3_client = boto3.client('s3')

In [ ]:
bucket = 'hbomax-datascience-development-dev'

In [ ]:
# collect column headers for the SHAP columns
shap_cols_df = pd.read_csv(f's3://{bucket}/free_trial_model/snowflake-hbomax-staging/schema/column_headers.csv',header=None)
shap_column_headers = list(shap_cols_df[shap_cols_df.columns[0]])

# include additional columns output by the batch transform job
shap_column_headers = ['PREDICTION','SHAP_EXPECTED'] + shap_column_headers

In [ ]:
# collect output files from the batch transform
response = s3_client.list_objects_v2(
        Bucket=bucket,
        Prefix ='free_trial_model/snowflake-hbomax-staging/results')
    
files = [r['Key'] for r in response['Contents'] if r['Size'] > 0]

raw_data = [ pd.read_csv(
        f's3://{bucket}/{file}', header=None) for file in files ]

concat_data = pd.concat(raw_data)

In [ ]:
# define the columns we want to keep from the original output (first column plus SHAP columns)
keep_cols = concat_data.columns[-len(shap_column_headers):].insert(loc=0, item=0)

In [ ]:
# create a DataFrame with the formatted output
shap_df = concat_data[keep_cols]

In [ ]:
# rename column headers
shap_df.columns = ['HBO_UUID'] + shap_column_headers

In [ ]:
# formatted output DataFrame 
shap_df.head()